In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from PIL import Image
import torchvision.models as models


import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets

import shutil

import copy
import os
import numpy as np

import itertools
import torch
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import datetime
from time import time

import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset

import wandb


torch.set_grad_enabled(True)
%matplotlib inline

from sklearn.metrics import confusion_matrix
import seaborn as sn
import itertools

import cv2

In [ ]:
# We use wandb to log our results
wandb.login()

# 1. Data preprocesing
Read the execution parameters. Preprocess the image ONLY if it's the first run. Construct the dataset and run an example image.

In [ ]:
first_run = False
num_run = 101
epochs = 50
learning_rate= 1e-6
N_images = 1584
batch_size = 18
torch.backends.cudnn.benchmark = True

In [ ]:
# If there is GPU, then use GPU.
import torch
if(torch.cuda.is_available()):
    print("SI")
else:
    print("NO")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

def load_ben_color(image, sigmaX=10):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (300, 300))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

In [ ]:
# Preprocessing of images. Transformations.
# Do it only once.
if (first_run):
    image_data = pd.read_csv('dataset_prova/dataset_prova_dades_imatges.csv', sep = ';', encoding="latin-1")
    transform = lambda y: transforms.Compose([ transforms.CenterCrop(y), transforms.Resize([300,300])]) 

    for i in range(0,N_images):
        ID_image,label = image_data.loc[i,['ID_Imatge','Retinopatia']]
        path = f"dataset_prova/classe_{label}/{ID_image}"
        image = Image.open(path)
        W, H = image.size
        image = transform(H)(image)
        image = np.array(image)
        image = load_ben_color(image)
        image = Image.fromarray(image)
        image.save(f"dataset_processat/classe_{label}/{ID_image}")
        print(f"dataset_processat/classe_{label}/{ID_image}")

In [ ]:
class Diabetic_Retinopathy_Dataset(Dataset): 
  def __init__(self):
    self.image_data = pd.read_csv('dataset_prova/dataset_prova_dades_imatges.csv', sep = ';', encoding="latin-1")
    self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(0,1)]) 

  def __getitem__(self, i):
    ID_imatge, label = self.image_data.loc[i,['ID_Imatge','Retinopatia']]
    path = f"dataset_processat/classe_{label}/{ID_imatge}"
    image = Image.open(path)
    image = self.transform(image)
    return image, label-1

  def __len__(self):
    return len(self.image_data)

In [ ]:
dataset__training, dataset_validation = torch.utils.data.random_split(Diabetic_Retinopathy_Dataset(), [round(N_images*0.8),round(N_images*0.2)]) 

dataloader_training = DataLoader(dataset__training, batch_size=batch_size, shuffle=True, pin_memory= True)
dataloader_validating = DataLoader(dataset_validation, batch_size=batch_size, shuffle=False, pin_memory= True)
x, labels = next(iter(dataloader_training))
print(x.shape)
print(labels.shape)

In [ ]:
image_data = pd.read_csv('dataset_prova/dataset_prova_dades_imatges.csv', sep = ';', encoding="latin-1")

In [ ]:
i = 964
Id, y = image_data.loc[i,['ID_Imatge','Retinopatia']]
path = f"dataset_processat_full/classe_{y}/{Id}"
print("Class:",y)
Image.open(path)


In [ ]:
dataset__training, dataset_validation = torch.utils.data.random_split(Diabetic_Retinopathy_Dataset(), [round(N_images*0.8),round(N_images*0.2)]) 

dataloader_training = DataLoader(dataset__training, batch_size=batch_size, shuffle=True, pin_memory= True)
dataloader_validating = DataLoader(dataset_validation, batch_size=batch_size, shuffle=False, pin_memory= True)
x, labels = next(iter(dataloader_training))
print(x.shape)
print(labels.shape)

# 2. Construct the model

Use a Resnet34 pretrained with imagenet data.

In [ ]:
def prediction2label(pred: np.ndarray):
    # Convert ordinal predictions to class labels
    return (pred > 0.5).cumprod(axis=1).sum(axis=1) - 1

In [ ]:
criterion_ord = nn.MSELoss(reduction='none')
def ordinal_regression(predictions, targets):
    # Ordinal regression with encoding as in https://arxiv.org/pdf/0704.1028.pdf

    # Create out modified target with [batch_size, num_labels] shape
    modified_target = torch.zeros_like(predictions) # vector de 0

    # Fill in ordinal target function, i.e. 0 -> [1,0,0,...]
    for i, target in enumerate(targets):
        modified_target[i,0:target+1] = 1

    return criterion_ord(predictions,modified_target).sum()

In [ ]:
# Take the ResNet34 model
model = models.resnet18(pretrained = True).to(device)

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs,5),
    nn.Sigmoid()
)

model = model.to(device)

optimizer = optim.Adam(model.parameters(), learning_rate)
criterion = ordinal_regression

In [ ]:
def adjust_learning_rate(optimizer, epoch, original_lr):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = original_lr * (0.1 ** (epoch // 30))
    # For some models, different parameters are in different groups with different lr
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def correct_predictions(predicted_batch, label_batch):
  pred = prediction2label(predicted_batch)
  acum = pred.eq(label_batch.view_as(pred)).sum().item()
  return acum

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    
    # save an extra copy if it is the best model yet
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')  

In [ ]:
classes = ('1: Normal','2: RD Lleu','3: RD Moderada','4: RD Severa','5: Proliferativa')
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    fig = plt.figure(figsize=(8,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    return fig

In [ ]:
def train(train_loader, model, criterion, optimizer, device):

    # Switch to train mode
    model.train()
    loss_mean = []
    acc_mean = []

    for i, (images, target) in enumerate(train_loader):
        # Reset gradients
        optimizer.zero_grad()

        #move images to gpu
        images = images.to(device)
        target = target.to(device)

        # Compute output
        output = model(images)

        loss = criterion(output,target)

        acc = 100 * (correct_predictions(output, target) / images.shape[0])
        
        loss.backward()

        loss_mean.append(loss.item())
        acc_mean.append(acc)

        optimizer.step()

    return np.mean(acc_mean), np.mean(loss_mean)

In [ ]:
def validate(val_loader, model, criterion, device):

    # Switch to evaluate mode
    model.eval()

    # We will save the values of the accuracies in this list to return the mean of the whole dataset at the end
    acc = 0
    loss = []

    y_pred = []
    y_true = []

    with torch.no_grad():  # We do not need to compute gradients
        for i, (images, target) in enumerate(val_loader):

            images = images.to(device)
            target = target.to(device)

            output = model(images)

            # loss
            loss.append(criterion(output, target).item())

            # measure accuracy
            acc += correct_predictions(output, target)

            # confusion matrix
            output = prediction2label(output).data.cpu().numpy()
            y_pred.extend(output) # Save Prediction
            
            target = target.data.cpu().numpy()
            y_true.extend(target) # Save Truth

    cf_matrix = confusion_matrix(y_true, y_pred)

    eval_acc = 100. * acc / len(val_loader.dataset)

    return eval_acc, np.mean(loss), cf_matrix

# 3.Execució

In [ ]:
wandb.finish() # This is needed just in case there was a wandb run from a previous execution
wandb.init(project="mnist_colab")
wandb.run.name = f'run_{num_run}'
best_acc = 0.
for epoch in range(epochs):
    adjust_learning_rate(optimizer, epoch, learning_rate)

    acc_tr, loss_tr = train(dataloader_training, model, criterion, optimizer, device)

    wandb.log({'Training/accuracy':acc_tr}, epoch)
    wandb.log({'Training/loss':loss_tr}, epoch)

    acc, loss, cf_matrix = validate(dataloader_validating, model, criterion,device)

    is_best = acc > best_acc
    best_acc1 = max(acc, best_acc)

    wandb.log({'Validation/accuracy': acc},epoch)
    wandb.log({'Validation/loss':loss},epoch)
    wandb.log({'Conf_matrix':plot_confusion_matrix(cf_matrix, classes,normalize=True)}, epoch) 

    print("Iteracio", epoch,"completada. Validate loss", loss, "train loss", loss_tr, "Validate acc", acc, "train acc", acc_tr)

    save_checkpoint({
        'epoch': epoch + 1,
        'arch': "resnet34",
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)


wandb.finish()
